<a href="https://colab.research.google.com/github/devulapallia1/Phishing/blob/main/592ML_keras_phishing_encode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
import numpy as np
import pandas as pd

TensorFlow version: 2.12.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # mount my Google Drive on runtime

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [ ]:
# same as assignment 2, dataset is from:
# www.kaggle.com/code/mohamedgalal1/phishing-detection-using-ml-techniques/data
# There are 507,195 urls in this dataset with 28% bad (/phishing) urls:

# specify the path and file name:
file = 'drive/MyDrive/Colab Notebooks/.../phishing_site_urls.csv' # <- you need to use the file location in your Google drive
data = pd.read_csv(file)
print(data.head())  # print to understand the dataset

In [ ]:
# data preprocessing:

# replace labels
data['Label'] = data['Label'].replace({'bad': 0, 'good': 1})

# remove duplicates
data.drop(data[data.URL.duplicated() == True].index, axis=0, inplace=True)
data.reset_index(drop=True)

In [ ]:
print(data.head())  # print to see the changes

                                                 URL  Label
0  nobell.it/70ffb52d079109dca5664cce6f317373782/...      0
1  www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...      0
2  serviciosbys.com/paypal.cgi.bin.get-into.herf....      0
3  mail.printakid.com/www.online.americanexpress....      0
4  thewhiskeydregs.com/wp-content/themes/widescre...      0


In [ ]:
# reference: https://towardsdatascience.com/how-to-preprocess-character-level-text-with-keras-349065121089

# Convert characters in URLs to integers:

tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(data['URL'])
data['encoded']= tokenizer.texts_to_sequences(data['URL'])  # add a column 'encoded' that converts url into a squence of integers

In [ ]:
tokenizer.word_index  # you can see the vocabulary: 'e'->1, 'o'->2, '/' ->5, '.' ->12, 'w'->21

In [ ]:
print(data.head())  # you will see the encoded column that shows each URL is now a sequence of integers

In [ ]:
# convert x (a list of integers) into a 2D array
def to2D(x):
  N = 400
  dim = int(N/2) # my image will be 2 * N/2. You should change it to a different dimension based on experiments
  size = len(x)
  if (size > N):
    size = N
  image = np.zeros([N])  # start from zeros
  for i in range(0, size):
    image[i] = x[i]
  image = image.reshape(2, dim)
  return image

In [ ]:
# convert the encoded column, which is a list, into a 2D array
# my image is 2 * someValue in the function to2D. You should change it based on experiments.

data['image'] = data['encoded'].apply(lambda x : to2D(x)) # x is a list, which cannot be re-shaped. Need to use np for help

In [ ]:
print(data.head())  # you will see the image column that shows each URL is now a 2D array

In [ ]:
# machine learing is to find a function f such that y = f(X), where X is data and y is its label
# get X and y ready

X = np.array(data['image'].tolist())
y = data['Label'].astype('float32')

In [ ]:
# the original dataset is too big. Choose a small percentage for X_train, y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [ ]:
# define your CNN model
# using keras functional API
yourModel= tf.keras.Sequential()
# add layers onto your model:

# compile your model
yourModel.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

# train the model and store the training history:
history = yourModel.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=64)  # use 20% of training set for validation set

